In [ ]:
%pip install mediapipe opencv-python numpy 

The code in below cell provides only the json file and does not show the live preview of the detection being processed, that functionality is added in the cell below.

In [ ]:
import cv2
import mediapipe as mp
import json
import numpy as np

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=2, enable_segmentation=False)
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture('5 2.MOV')

correct_form = []

def calculate_angle(a, b, c):
    """Calculate the angle between three points a, b, c."""
    a = np.array([a.x, a.y, a.z])
    b = np.array([b.x, b.y, b.z])
    c = np.array([c.x, c.y, c.z])
    
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        frame_data = {'landmarks': [], 'angles': {}}
        
        # Extract landmarks
        landmarks = results.pose_landmarks.landmark
        for landmark in landmarks:
            frame_data['landmarks'].append({
                'x': landmark.x,
                'y': landmark.y,
                'z': landmark.z
            })
        # Calculate joint angles
        frame_data['angles']['left_elbow'] = calculate_angle(
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
            landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
        )
        frame_data['angles']['right_elbow'] = calculate_angle(
            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
            landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        )
        frame_data['angles']['left_knee'] = calculate_angle(
            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
            landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
        )
        frame_data['angles']['right_knee'] = calculate_angle(
            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
            landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
        )


        # Add the frame data to correct form
        correct_form.append(frame_data)

    # Display the video frame with landmarks
    cv2.imshow('Capture Correct Form', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Save the JSON file
with open('5 2.json', 'w') as json_file:
    json.dump(correct_form, json_file, indent=4)

print("Correct form with joint angles saved to 'json/5 2.json'")


Correct form with joint angles saved to 'json/5 2.json'


In the below cell there is the code to check the angles for full body, if the client can state if some of the exercises will be seating only or standing only different versions of code can be used to get the marking data (json file) for better performance, the code in the above cell measures only upper body angles 

In [13]:
import cv2
import mediapipe as mp
import json
import numpy as np

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=2, enable_segmentation=False)
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture('5 2.MOV')

correct_form = []

def calculate_angle(a, b, c):
 
    a = np.array([a.x, a.y, a.z])
    b = np.array([b.x, b.y, b.z])
    c = np.array([c.x, c.y, c.z])
    
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        frame_data = {'landmarks': [], 'angles': {}}
        
        # Extract landmarks
        landmarks = results.pose_landmarks.landmark
        for landmark in landmarks:
            frame_data['landmarks'].append({
                'x': landmark.x,
                'y': landmark.y,
                'z': landmark.z
            })
        
        # Calculate joint angles
        neck_angle = calculate_angle(
            landmarks[mp_pose.PoseLandmark.NOSE.value],
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        )
        frame_data['angles']['neck'] = neck_angle

        left_torso_angle = calculate_angle(
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
        )
        frame_data['angles']['left_torso'] = left_torso_angle

        right_torso_angle = calculate_angle(
            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
        )
        frame_data['angles']['right_torso'] = right_torso_angle

        left_hip_angle = calculate_angle(
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
        )
        frame_data['angles']['left_hip'] = left_hip_angle

        right_hip_angle = calculate_angle(
            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
        )
        frame_data['angles']['right_hip'] = right_hip_angle

        left_leg_straightness_angle = calculate_angle(
            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
            landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
        )
        frame_data['angles']['left_leg_straightness'] = left_leg_straightness_angle

        right_leg_straightness_angle = calculate_angle(
            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
            landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
        )
        frame_data['angles']['right_leg_straightness'] = right_leg_straightness_angle

        left_arm_angle = calculate_angle(
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
            landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
        )
        frame_data['angles']['left_arm'] = left_arm_angle

        right_arm_angle = calculate_angle(
            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
            landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        )
        frame_data['angles']['right_arm'] = right_arm_angle

        # Add the frame data to correct form
        correct_form.append(frame_data)

        # Draw landmarks on the frame
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Display angles on the frame
        # Define starting position for text display
        y_offset = 50

        # Display the angles
        for joint, angle in frame_data['angles'].items():
            cv2.putText(frame, f'{joint}: {angle:.2f}°', (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            y_offset += 30  # Move the y position for the next angle text

    # Display the video frame with landmarks and angles
    cv2.imshow('Capture Correct Form', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Save the JSON file
with open('5 2.json', 'w') as json_file:
    json.dump(correct_form, json_file, indent=4)

print("Correct form with joint angles saved to 'json/5 2.json'")


Correct form with joint angles saved to 'json/5 2.json'
